This uses the example Chinook database.
To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the `.db` file in a notebooks folder at the root of this repository.

In [1]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [2]:
db = SQLDatabase.from_uri("sqlite:////Users/harrisonchase/Downloads/chembl_31/chembl_31_sqlite/chembl_31.db")
llm = OpenAI(temperature=0)
db_chain = SQLDatabaseChain(llm=llm, database=db)

In [18]:
db.n = 43

In [20]:
db_chain.query("Which tables should I look at to determine how many cruzi-pain inhibiting compounds are there?")

 SELECT COUNT(DISTINCT molregno)
FROM activities
WHERE standard_type = 'EC50'
AND standard_units = 'uM'
AND relation = '='
AND value <= 100
[(0,)]


' 0'

In [6]:
print(db.table_info)

The 'action_type' table has columns: action_type (VARCHAR(50)), description (VARCHAR(200)), parent_type (VARCHAR(50)).
The 'activities' table has columns: activity_id (BIGINT), assay_id (BIGINT), doc_id (BIGINT), record_id (BIGINT), molregno (BIGINT), standard_relation (VARCHAR(50)), standard_value (NUMERIC), standard_units (VARCHAR(100)), standard_flag (SMALLINT), standard_type (VARCHAR(250)), activity_comment (VARCHAR(4000)), data_validity_comment (VARCHAR(30)), potential_duplicate (SMALLINT), pchembl_value (NUMERIC(4, 2)), bao_endpoint (VARCHAR(11)), uo_units (VARCHAR(10)), qudt_units (VARCHAR(70)), toid (INTEGER), upper_value (NUMERIC), standard_upper_value (NUMERIC), src_id (INTEGER), type (VARCHAR(250)), relation (VARCHAR(50)), value (NUMERIC), units (VARCHAR(100)), text_value (VARCHAR(1000)), standard_text_value (VARCHAR(1000)).
The 'activity_properties' table has columns: ap_id (BIGINT), activity_id (BIGINT), type (VARCHAR(250)), relation (VARCHAR(50)), value (NUMERIC), units (